![Vespa logo](https://vespa.ai/assets/vespa-logo-color.png)

# Pyvespa examples

This is a notebook with short examples one can build applications from.

Refer to [troubleshooting](https://pyvespa.readthedocs.io/en/latest/troubleshooting.html), which also has utilies for debugging.

In [ ]:
!pip3 install pyvespa

## Farthest neighbor

_Find the point that is  **most**  distant from a point,
using  [euclidean-distance](https://docs.vespa.ai/en/reference/ranking-expressions.html#euclidean-distance-t),
in 3D space_

This is a simple example feeding documents with a tensor representing a point in space,
and a rank profile calculating the distance between a point in the query and the point in the documents.

Deploy the Application Package:

In [6]:
from vespa.package import ApplicationPackage, Field, RankProfile
from vespa.deployment import VespaDocker

app_package = ApplicationPackage(name="distantneighbor")

app_package.schema.add_fields(
    Field(name = "point", type = "tensor<float>(d[3])", indexing = ["attribute", "summary"])
)

app_package.schema.add_rank_profile(
    RankProfile(
        name = "max_distance",
        inputs = [("query(qpoint)", "tensor<float>(d[3])")],
        first_phase = "euclidean_distance(attribute(point), query(qpoint), d)"
    )
)

vespa_docker = VespaDocker()
app = vespa_docker.deploy(application_package=app_package)

Waiting for configuration server, 0/300 seconds...
Waiting for configuration server, 5/300 seconds...
Waiting for application status, 0/300 seconds...
Waiting for application status, 5/300 seconds...
Finished deployment.


Feed points in 3d space using a 3-dimensional [indexed tensor](https://docs.vespa.ai/en/tensor-user-guide.html).
Pyvespa feeds using the [/document/v1/ API](https://docs.vespa.ai/en/reference/document-v1-api-reference.html), refer to [document format](https://docs.vespa.ai/en/reference/document-json-format.html):

In [7]:
feed = [
        {
            'id': 0,
            'fields': {
                'point': [0.0, 1.0, 2.0]
            }
        },
        {
            'id': 1,
            'fields': {
                'point': [1.0, 2.0, 3.0]
            }
        },
        {
            'id': 2,
            'fields': {
                'point': [2.0, 3.0, 4.0]
            }
        }
]

response = app.feed_batch(schema="distantneighbor", batch=feed, asynchronous=False)

Successful documents fed: 3/3.
Batch progress: 1/1.


Query from origo using [YQL](https://docs.vespa.ai/en/query-language.html).
The rank profile will rank the most distant points highest, here `sqrt(2*2 + 3*3 + 4*4) = 5.385`:

In [15]:
import json
result = app.query(body={
  'yql': 'select point from distantneighbor where true',
  'input.query(qpoint)': '[0.0, 0.0, 0.0]',
  'ranking.profile': 'max_distance'
})
print(json.dumps(result.hits, indent=4))

[
    {
        "id": "index:distantneighbor_content/0/c81e728dbd4b9a8f48e718f4",
        "relevance": 5.385164807134504,
        "source": "distantneighbor_content",
        "fields": {
            "point": {
                "type": "tensor<float>(d[3])",
                "values": [
                    2.0,
                    3.0,
                    4.0
                ]
            }
        }
    },
    {
        "id": "index:distantneighbor_content/0/c4ca42382215f2bce77b5d97",
        "relevance": 3.7416573867739413,
        "source": "distantneighbor_content",
        "fields": {
            "point": {
                "type": "tensor<float>(d[3])",
                "values": [
                    1.0,
                    2.0,
                    3.0
                ]
            }
        }
    },
    {
        "id": "index:distantneighbor_content/0/cfcd20848a3f84e145bdb949",
        "relevance": 2.23606797749979,
        "source": "distantneighbor_content",
        "fields": {
 

Query from `[1.0, 2.0, 2.9]` - find that `[2.0, 3.0, 4.0]` is most distant:

In [14]:
result = app.query(body={
  'yql': 'select point from distantneighbor where true',
  'input.query(qpoint)': '[1.0, 2.0, 2.9]',
  'ranking.profile': 'max_distance'
})
print(json.dumps(result.hits, indent=4))

[
    {
        "id": "index:distantneighbor_content/0/c81e728dbd4b9a8f48e718f4",
        "relevance": 1.7916472308265357,
        "source": "distantneighbor_content",
        "fields": {
            "point": {
                "type": "tensor<float>(d[3])",
                "values": [
                    2.0,
                    3.0,
                    4.0
                ]
            }
        }
    },
    {
        "id": "index:distantneighbor_content/0/cfcd20848a3f84e145bdb949",
        "relevance": 1.6763055154708881,
        "source": "distantneighbor_content",
        "fields": {
            "point": {
                "type": "tensor<float>(d[3])",
                "values": [
                    0.0,
                    1.0,
                    2.0
                ]
            }
        }
    },
    {
        "id": "index:distantneighbor_content/0/c4ca42382215f2bce77b5d97",
        "relevance": 0.09999990575011103,
        "source": "distantneighbor_content",
        "fields":

## Cleanup

In [5]:
vespa_docker.container.stop()
vespa_docker.container.remove()